![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 3.6: Generators: Types
#### Written by Stevo Bailey ([stevo@berkeley.edu](mailto:stevo@berkeley.edu)) Adam Izraelivitz ([azidar@berkeley.edu](mailto:stevo@berkeley.edu)) Chick Markley ([chick@berkeley.edu](mailto:chick@berkeley.edu))

## Table of Contents
Scala is a strongly typed language.
Type system, type parameterization, clonetype? (seen before in 3.1, but re-explain here)

- [Scala vs. Chisel types](#scala-vs-chisel-types)
- [Type Coercion](#coercion)
- [asInstanceOf](#asInstanceOf)
- [Type Matching](#type-matching)
- [Type Safe Connections?](#type-safe-connections)
- [Type Generics](#type-generics)
- [Example: Type-generic ShiftRegister](#type-generic-shift-register)

- [Creating a custom type](#creating-a-custom-type)
  - DSPcomplex
  - Sign-magnitude (+1 from Paul)
  - One’s complement?


## Setup

In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19` 
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester, TesterOptionsManager}
import chisel3.util._

// Convenience function to invoke Chisel and grab emitted Verilog.
def getVerilog(args: Array[String] = Array(), dut: () => chisel3.core.UserModule): String = {
  import firrtl._
  return chisel3.Driver.execute(args, dut) match {
    case s:ChiselExecutionSuccess => s.firrtlResultOption match {
      case Some(f:FirrtlExecutionSuccess) => f.emitted
    }
  }
}


# Types in Scala<a name="types-in-scala"></a>
Scala functions were introduced in Module 1. Functions take any number of inputs and produce one output. Inputs are often called arguments to a function. To produce no output, return the `Unit` type. Below are some examples of functions.

## Scala vs. Chisel Types<a name="scala-vs-chisel-types"></a>

## Coercion name<a name="type-coercion"></a>
### asInstanceOf
### getTypeOf

## Type Matching<a name="type-matching"></a>
### match operator
### partial functions
### unapply

## Type Safe Connections<a name="type-safe-connections"></a>

## Type Generics<a name="type-generics"></a>

## Example: Type Generic ShiftRegister<a name="type-generic-shift-register"></a>

>Avoid using inheritance with type generics, this is very tricky to do properly, and can get frustrating quickly

In Scala, objects and functions aren't the only things we can treat as parameters. We can also treat types as parameters.
We usually need to provide a type constraint.
In this case, we want to be able to put objects in a bundle, connect (:=) them, and create registers with them (RegNext).
These operations cannot be done on arbitrary objects; for example wire := 3 is illegal because 3 is a Scala Int, not a Chisel UInt.
If we use a type constraint to say that type T is a subclass of Data, then we can use := on any objects of type T because := is defined for all Data.
Here is an implementations of a simple shift register that take types as a parameter.
*gen* is an argument of type T that tells what width to use, for example new ShiftRegister(UInt(4.W)) is a shift register for 4-bit UInts.
*gen* also allows the Scala compiler to infer the type T- you can write new ShiftRegister[UInt](UInt(4.W)) if you want to to be more specific, but the Scala compiler is smart enough to figure it out if you leave out the [UInt].

In [ ]:
class ShiftRegisterIO[T <: Data](gen: T, n: Int) extends Bundle {
    require (n >= 0, "Shift register must have non-negative shift")
    
    val in = Input(gen.cloneType)
    val out = Output(Vec(n + 1, gen.cloneType)) // + 1 because in is included in out
}

class ShiftRegister[T <: Data](gen: T, n: Int) extends Module {
    val io = IO(new ShiftRegisterIO(gen, n))
    
    io.out.foldLeft(io.in) { case (in, out) =>
        out := in
        RegNext(in)
    }
}

class ShiftRegisterTester[T <: Bits](c: ShiftRegister[T]) extends PeekPokeTester(c) {
    println(s"Testing ShiftRegister of type ${c.io.in} and depth ${c.io.out.length}")
    for (i <- 0 until 10) {
        poke(c.io.in, i)
        println(s"$i: ${peek(c.io.out)}")
        step(1)
    }
}

Driver(() => new ShiftRegister(UInt(4.W), 5)) { c => new ShiftRegisterTester(c) }
Driver(() => new ShiftRegister(SInt(6.W), 3)) { c => new ShiftRegisterTester(c) }

## Creating a Custom Type<a name="creating-a-custom-type"></a>

### DspComplex
### Sign-magnitude Numbers